In [73]:
import pandas as pd
import pdfplumber
import requests

# URLs of the PDF files
urls = [
    'https://heartlandforward.org/wp-content/uploads/2020/12/Most-Dynamic-Metros.pdf',
    'https://heartlandforward.org/wp-content/uploads/2022/04/Most-Dynamic-Metros_9.pdf',
    'https://heartlandforward.org/wp-content/uploads/2023/03/MostDyanmicMetros2022.pdf'
]

# Pages to extract for each PDF
pages_list = [range(134, 149), range(23, 34), range(31, 41)]

# Starting year
year = 2019

# Initialize an empty DataFrame for the final merged result
final_merged_df = None

# Loop through each URL and corresponding page ranges
for url, pages in zip(urls, pages_list):
    # Send a GET request to fetch the content of the PDF
    response = requests.get(url)
    response.raise_for_status()

    # Save the PDF locally
    with open('Most-Dynamic-Metros.pdf', 'wb') as f:
        f.write(response.content)

    # Open the PDF and extract tables
    with pdfplumber.open('Most-Dynamic-Metros.pdf') as pdf:
        all_tables = []
        for page_num in pages:
            page = pdf.pages[page_num - 1]
            page_tables = page.extract_tables()
            for table in page_tables:
                # Convert each table to a DataFrame
                df = pd.DataFrame(table[1:], columns=table[0])
                all_tables.append(df)

    # Concatenate all DataFrames
    year_df = pd.concat(all_tables, ignore_index=True)

    # Data cleaning
    year_df = year_df.replace('\n', ' ', regex=True)
    year_df = year_df.replace(' ', '', regex=True)
    year_df = year_df.replace('-', '', regex=True)
    year_df = year_df.iloc[:, :2].rename(columns={'': 'Metropolitan Statistical Area', 'Overall\nRanking': f'{year}Rank',
                                                  'OVERALL\nRANKING': f'{year}Rank'})

    # Merge with the final DataFrame
    if final_merged_df is None:
        final_merged_df = year_df
    else:
        final_merged_df = pd.merge(final_merged_df, year_df, on='Metropolitan Statistical Area', how='outer')

    # Increment year
    year += 1

# Output the final merged DataFrame
print(final_merged_df)


     Metropolitan Statistical Area 2019Rank 2020Rank 2021Rank
0                       Midland,TX        1       38        6
1    SanJoseSunnyvaleSantaClara,CA        2        1        1
2                     ProvoOrem,UT        3        5        9
3                       Boulder,CO        4        9        7
4    SanFranciscoOaklandHayward,CA        5      NaN      NaN
..                             ...      ...      ...      ...
418                   Fairbanks,AK      NaN      313      371
419                   Anchorage,AK      NaN      326      350
420          KingsportBristol,TNVA      NaN      351      240
421             CapeGirardeau,MOIL      NaN      NaN         
422                 St.Joseph,MOKS      NaN      NaN         

[423 rows x 4 columns]


In [6]:
import pandas as pd

def extract_state(metro_name):
    """Extract the state abbreviation from a metro name."""
    return metro_name.split(',')[-1].strip() if ',' in metro_name else None

def have_common_substring(str1, str2, min_length=5):
    """Check if two strings share a common significant substring."""
    for i in range(len(str1)):
        for j in range(min_length, len(str1) - i + 1):
            substring = str1[i:i+j]
            if substring in str2 and len(substring) >= min_length and len(substring)/len(str2) > 0.5:
                return True
    return False

# Load the dataset
file_path = './Most-Dynamic-Metros.csv'
df = pd.read_csv(file_path)

# Identify columns that contain the ranks
rank_columns = [col for col in df.columns if 'Rank' in col]

# Add a column to mark matched rows
df['Matched'] = False

# Iterate over the DataFrame to find matches
for i, row1 in df.iterrows():
    for j, row2 in df.iterrows():
        if i != j:
            # Check for common substring and same state
            if have_common_substring(row1['Metropolitan Statistical Area'], row2['Metropolitan Statistical Area']) \
               and extract_state(row1['Metropolitan Statistical Area']) == extract_state(row2['Metropolitan Statistical Area']):
                # Fill in missing ranks and mark as matched
                is_filled = False
                for col in rank_columns:
                    if pd.isna(row1[col]) and not pd.isna(row2[col]):
                        df.at[i, col] = row2[col]
                        is_filled = True
                if is_filled:
                    df.at[j, 'Matched'] = True

# Remove marked duplicate rows
df = df[~df['Matched']]

# Drop the 'Matched' column as it's no longer needed
df.drop(columns=['Matched'], inplace=True)

# Save the updated DataFrame
df.to_csv('./Updated-Most-Dynamic-Metros.csv', index=False)


In [92]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Load the dataset
file_path = './Most-Dynamic-Metros.csv'
metros_df = pd.read_csv(file_path)

# Custom function to find the closest match with a different scorer and lower threshold
def get_closest_match(x, list_strings, scorer=fuzz.partial_ratio, threshold=0):
    if x in list_strings:  # Skip if the exact match is in the list
        return None
    best_match = process.extractOne(x, list_strings, scorer=scorer)
    return best_match[0] if best_match and best_match[1] > threshold else None

# List of unique strings for matching
list_strings = df['Metropolitan Statistical Area'].unique()

# Apply the matching function
df['Closest Match'] = df['Metropolitan Statistical Area'].apply(lambda x: get_closest_match(x, list_strings))

# Display the first few rows of the DataFrame to understand the matches
print(df[['Metropolitan Statistical Area', 'Closest Match']].head(10))

   Metropolitan Statistical Area Closest Match
0                     Midland,TX          None
1  SanJoseSunnyvaleSantaClara,CA          None
2                   ProvoOrem,UT          None
3                     Boulder,CO          None
4  SanFranciscoOaklandHayward,CA          None
5             AustinRoundRock,TX          None
6       SeattleTacomaBellevue,WA          None
7                     Greeley,CO          None
8  NaplesImmokaleeMarcoIsland,FL          None
9                   St.George,UT          None


In [70]:
import pandas as pd
import pdfplumber
import requests
from fuzzywuzzy import process

# Define the fuzzy matching function
def fuzzy_merge(df1, df2, key1, key2, threshold=90, limit=1):
    """
    Fuzzy merges two DataFrames.
    """
    # Create a mapping from the key1 values to the best matching key2 values
    mapping = df1[key1].apply(lambda x: process.extractOne(x, df2[key2], score_cutoff=threshold))

    # Extract the matched key2 values and scores into separate columns
    df1['matched_key'] = mapping.apply(lambda x: x[0] if x else None)
    df1['score'] = mapping.apply(lambda x: x[1] if x else None)

    # Perform the merge using the new 'matched_key' column in df1 and key2 in df2
    return pd.merge(df1, df2, left_on='matched_key', right_on=key2, how='left')

# URLs and page ranges of the PDF files
urls_pages = {
    'https://heartlandforward.org/wp-content/uploads/2020/12/Most-Dynamic-Metros.pdf': range(134, 149),
    'https://heartlandforward.org/wp-content/uploads/2022/04/Most-Dynamic-Metros_9.pdf': range(23, 34),
    'https://heartlandforward.org/wp-content/uploads/2023/03/MostDyanmicMetros2022.pdf': range(31, 41)
}

# Initialize an empty DataFrame for the final merged result
final_df = pd.DataFrame()

# Process each PDF
for year, (url, pages) in enumerate(urls_pages.items(), start=2019):
    response = requests.get(url)
    response.raise_for_status()

    with open('Most-Dynamic-Metros.pdf', 'wb') as f:
        f.write(response.content)

    with pdfplumber.open('Most-Dynamic-Metros.pdf') as pdf:
        all_tables = []
        for page_num in pages:
            page = pdf.pages[page_num - 1]
            page_tables = page.extract_tables()
            for table in page_tables:
                df = pd.DataFrame(table[1:], columns=table[0])
                all_tables.append(df)

    year_df = pd.concat(all_tables, ignore_index=True)
    year_df = year_df.replace('\n', ' ', regex=True).replace(' ', '', regex=True).replace('-', '', regex=True)
    year_df = year_df.iloc[:, :2].rename(columns={'': 'Metropolitan Statistical Area', 'Overall\nRanking': f'{year}Rank', 'OVERALL\nRANKING': f'{year}Rank'})

    # Merge with the final DataFrame using fuzzy matching
    if final_df.empty:
        final_df = year_df
    else:
        final_df = fuzzy_merge(final_df, year_df, 'Metropolitan Statistical Area', 'Metropolitan Statistical Area')

# Save the final merged DataFrame
final_df.to_csv('Most-Dynamic-Metros-Merged.csv', index=False)


KeyError: 'Metropolitan Statistical Area'

In [57]:
import pandas as pd
import pdfplumber
import requests

# URL of the PDF file
url = 'https://heartlandforward.org/wp-content/uploads/2023/03/MostDyanmicMetros2022.pdf'

# Send a GET request to fetch the content of the PDF
response = requests.get(url)
response.raise_for_status()

# Save the PDF locally
with open('Most-Dynamic-Metros.pdf', 'wb') as f:
    f.write(response.content)

# Specify the range of pages
pages = range(31, 41)

# Open the PDF and extract tables
with pdfplumber.open('Most-Dynamic-Metros.pdf') as pdf:
    all_tables = []
    for page_num in pages:
        page = pdf.pages[page_num - 1]
        page_tables = page.extract_tables()
        for table in page_tables:
            # Convert each table to a DataFrame
            df = pd.DataFrame(table[1:], columns=table[0])
            all_tables.append(df)

final_df22 = pd.concat(all_tables, ignore_index=True)
final_df22 = final_df22.replace('\n', ' ', regex=True)
final_df22 = final_df22.replace(' ', '', regex=True)
final_df22 = final_df22.replace('-', '', regex=True)
final_df22 = final_df22.iloc[:, :2].rename(columns={'': 'Metropolitan Statistical Area', 'OVERALL\nRANKING': '2021Rank'})
final_df22.to_csv('Most-Dynamic-Metros.csv', index=False)

In [58]:
final_df22

,Metropolitan Statistical Area,2021Rank
0,"SanJoseSunnyvaleSantaClara,CA",1
1,"ElkhartGoshen,IN",2
2,"SanFranciscoOaklandBerkeley,CA",3
3,"AustinRoundRockGeorgetown,TX",4
4,"TheVillages,FL",5
...,...,...
379,"Farmington,NM",380
380,"BeaumontPortArthur,TX",381
381,"Enid,OK",382
382,"CapeGirardeau,MOIL",


In [59]:
mdms = pd.merge(final_df20, final_df21, on='Metropolitan Statistical Area', how='outer')
mdms = pd.merge(mdms, final_df22, on='Metropolitan Statistical Area', how='outer')



In [61]:
mdms.to_csv('Most-Dynamic-Metros.csv', index=False)